In [1]:
import os
import tifffile
import numpy as np
from lumicks import pylake

In [2]:
def create_output_folder(data_folder):
    """
    Create an output folder if it doesn't exist.

    :param data_folder: Path to the data folder.
    :param output_folder_name: Name of the output folder.
    :return: Path to the output folder.
    """
    output_folder = data_folder
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    return output_folder

def optimize_brightness_contrast(image, lower_percentile=1, upper_percentile=98):
    """
    Optimize the brightness and contrast of an image using percentiles.

    :param image: Input image.
    :param lower_percentile: Lower percentile for clipping, default is 1.
    :param upper_percentile: Upper percentile for clipping, default is 98.
    :return: Image with optimized brightness and contrast.
    """
    minval = np.percentile(image, lower_percentile)
    maxval = np.percentile(image, upper_percentile)
    image = np.clip(image, minval, maxval)
    image = ((image - minval) / (maxval - minval)) * 255
    return image.astype(np.uint8)

def save_kymographs_as_tiff(data_folder, marker_name):
    """
    Save kymographs as tiff files with optimized brightness and contrast.

    :param data_folder: Path to the data folder.
    :param marker_name: Marker.h5 file to process.
    """
    output_folder = create_output_folder(f'{data_folder}/tiffs/')
    marker = pylake.File(os.path.join(data_folder, marker_name))
    
    for kymo_ID in marker.kymos:
        kymo_duration = marker.kymos[kymo_ID].duration
        if kymo_duration < 35: continue
        kymo_img = marker.kymos[kymo_ID].get_image(channel='red')
        kymo_img = np.round(255 * kymo_img)
        kymo_img = optimize_brightness_contrast(kymo_img)
        
        file_save = f"{marker_name.removesuffix('.h5')}_kymo_{kymo_ID}.tiff"
        tifffile.imwrite(os.path.join(output_folder, file_save), kymo_img)


In [3]:
# Define the data folder path and dates to process
data_folder = '../test_folder/'
marker_name = '20241219-125005 Marker 1.h5'

# Save kymographs as tiff files with optimized brightness and contrast
save_kymographs_as_tiff(data_folder, marker_name)